In [ ]:
import copy

import cornac
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

from recommenders.utils.timer import Timer

from src.compressors import quantization
from src.metrics import get_model_size, calculate_metrics

In [12]:


MOVIELENS_DATA_SIZE = '1M'

# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 100
ENCODER_DIMS = [200]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 1024
LEARNING_RATE = 0.001

In [13]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

100%|██████████| 5.78k/5.78k [00:01<00:00, 3.93kKB/s]


In [14]:
train, test = python_random_split(data, 0.75)

In [15]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=1234)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 6040
Number of items: 3676


In [ ]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=1234,
    # use_gpu=torch.cuda.is_available(),
    use_gpu=False,
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
base_metrics = calculate_metrics(train, test, bivae)

In [ ]:
from torch.utils.mobile_optimizer import optimize_for_mobile
import torch.jit
from torch import Tensor, IntTensor, ByteTensor, LongTensor
from types import MethodType
from cornac.models.bivaecf.bivae import BiVAE

bivae2=copy.deepcopy(bivae.bivae)

def forward(self, user_idx):
    theta_u = self.mu_theta[user_idx].view(1, -1)
    beta = self.mu_beta
    known_item_scores = (
            self.decode_user(theta_u, beta).cpu()#.numpy().ravel()
        )

    return known_item_scores

bivae2.forward = MethodType(forward, bivae2)

example = (bivae2.mu_theta[1].view(1, -1), bivae2.mu_beta)
bivae2.eval()
# scripted_model = torch.jit.trace(bivae2, LongTensor([0]))
scripted_model = torch.jit.script(bivae2)
scripted_model._save_for_lite_interpreter("normal.pt")

optimized  = optimize_for_mobile(scripted_model)
optimized._save_for_lite_interpreter("opti.pt")

quantized_model=torch.quantization.convert(bivae2)
quantized = torch.jit.script(quantized_model)
quantized._save_for_lite_interpreter("quant.pt")

both  =  optimize_for_mobile(scripted_model)
both._save_for_lite_interpreter("both.pt")

In [ ]:
%%time
for _ in range(100000):
    scripted_model(LongTensor([0]))

In [ ]:
%%time
for _ in range(100000):
    quantized(LongTensor([0]))

In [ ]:
%%time
for _ in range(100000):
    optimized(LongTensor([0]))

In [ ]:
%%time
for _ in range(100000):
    both(LongTensor([0]))

In [22]:
# import torch.ao
#
# model_quatized = torch.ao.quantization.quantize_dynamic(
#     bivae.bivae,
#     {torch.nn.Linear},
#     dtype=torch.qint8
# )

In [ ]:
quantized_bivae = quantization.get_post_dynamic(bivae)
calculate_metrics(train, test, bivae)

In [41]:
# quantized_bivae.save("ckpts")

BiVAECF model is saved to ckpts\BiVAECF\2023-04-19_16-18-13-155663.pkl


'ckpts\\BiVAECF\\2023-04-19_16-18-13-155663.pkl'

In [39]:
# get_model_size(bivae.bivae, "original")

model:  original  	 Size (KB): 2394.711


2394711

In [40]:
# get_model_size(model_quatized, "quantized")

model:  quantized  	 Size (KB): 607.695


607695